So far we've covered decision trees growing with an entropy criterion. In doing so, however, we glossed over how that would actually work. In this assignment we'll give more detail into how an algorithm to do that would practically function.

Here we'll cover one popular algorithm for building a decision tree using entropy and information gain: **ID3**.

## The ID3 Algorithm

ID3 stands for _Iterative Dichotomizer 3_, and is one of the simplest ways to create a decision tree. It can be generalized into more robust scenarios, but the implementation is based on the framework we'll go over here. Essentially ID3 goes through every feature to find the most valuable attribute and then splits based on it. It moves further and further down the tree until it either has a pure class or has met a terminating condition. We'll cover the details below.

Before you can start working with ID3, however, there are some requirements for the data in this most basic form. Firstly, outcomes have to be binary. The simplest version of ID3 is a binary classifier. The attributes we're using to build the tree also have to be categorical. Attributes can have many categories but they must be known and countable.

If those two criteria are met then you can build a basic ID3 classifying algorithm.

The other thing we'll need for this is our definition of entropy. Recall from the previous assignment that we're going to use Shannon Entropy $H$, defined as:

$$ H = -\sum_{i=1}^n P(x_i)log_2 P(x_i) $$

For simplicity of calculation, we're actually going to do a slight transform on this definition. Recall from a (quite possibly long ago) algebra class that you can bring exponentials out to the front of a logarithm. In this case we'll raise the probability to -1, changing the formula to:

$$ H = \sum_{i=1}^n P(x_i)log_2 \frac{1}{P(x_i)} $$

This removes the negative sign up front and will make it easier for us to implement this formula.

## Calculating Entropy

Since this algorithm is based on entropy let's go over a quick example of how to calculate it.

Let's say we have 20 students, and we're trying to classify them as male and female. The only attribute we have is whether their height is tall, medium, or short. Of the 20 students, 12 are boys with and 8 are girls. Of the 12 boys, 4 are tall, 6 are medium and 2 are short. Of the 8 girls, 1 is tall, 2 are medium, and 5 are short.

What is the entropy, both before any rule is applied and then after applying a rule for being tall?

The initial entropy is just the formula plugged in over both the possible classes of interest:

$$ H = P(male)*log_2 \frac{1}{P(male)} + P(female)*log_2 \frac{1}{P(female)}  $$


$$ = \frac{12}{20}*log_2 \frac{20}{12} + \frac{8}{20}*log_2 \frac{20}{8} = .971 $$

What if we apply the rule _height = short_? We need to calculate the weighted average of the two entropies, one for the short students and one for the non-short students.

$$ H(short) = P(male)*log_2 \frac{1}{P(male)} + P(female)*log_2 \frac{1}{P(female)}  $$

$$ = \frac{2}{7}*log_2 \frac{7}{2} + \frac{5}{7}*log_2 \frac{7}{5} = .863 $$

$$ H(not\_short) = P(male)*log_2 \frac{1}{P(male)} + P(female)*log_2 \frac{1}{P(female)}  $$

$$ = \frac{10}{13}*log_2 \frac{13}{10} + \frac{3}{13}*log_2 \frac{13}{3} = .779 $$

Note that all the probabilities here are conditional on the criteria we're assuming (short or not short). Now the weighted average of the two is just:

$$ P(short) * H(short) + P(not\_short) * H(not\_short) = \frac{7}{20} * .863 + \frac{13}{20} * .779 = .809 $$

So our entropy from this question would go from .971 to .809. That's an improvement. Use the space below to calculate the entropy of the other criteria, for we asked whether the students were tall or medium.

In [3]:
from math import log2

# TALL students: 4 boys, 1 girl.
# Before:
H_tall = (4/5) * log2(5/4) + (1/5) * log2(5)
print("Probability of being tall, before entropy rule is applied: {}".format(H_tall))

# After:
H_not_tall = (8/15) * log2(15/8) + (7/15) * log2(15/7)
print("Probability of not being tall, before entropy rule is applied: {}".format(H_not_tall))

# Weighted average:
wa_tall = (5/20) * H_tall + (15/20) * H_not_tall
print("Weighted average of H(tall) and H(not tall): {}".format(wa_tall))

Probability of being tall, before entropy rule is applied: 0.7219280948873623
Probability of not being tall, before entropy rule is applied: 0.9967916319816366
Weighted average of H(tall) and H(not tall): 0.9280757477080679


In [5]:
# Medium students: 6 boys, 2 girls.

# Before:
H_medium = (6/8) * log2(8/6) + (2/8) * log2(8/2)
print("Probability of being medium, before entropy rule is applied: {}".format(H_medium))

# After:
H_not_medium = (6/12) * log2(12/6) + (6/12) * log2(12/6)
print("Probability of not being medium, before entropy rule is applied: {}".format(H_not_medium))

# Weighted average:
wa_medium = (8/20) * H_medium + (12/20) * H_not_medium
print("Weighted average of H(medium) and H(not medium): {}".format(wa_medium))

Probability of being medium, before entropy rule is applied: 0.8112781244591327
Probability of not being medium, before entropy rule is applied: 1.0
Weighted average of H(medium) and H(not medium): 0.9245112497836531


In [2]:
# EXAMPLE SOLUTION

# Tall.
H_tall = 4 / 5 * log2(5 / 4) + 1 / 5 * log2(5 / 1)
H_not_tall = 8 / 15 * log2(15 / 8) + 7 / 15 * log2(15 / 7)

entropy_tall = 5 / 20 * H_tall + 15 / 20 * H_not_tall


# Medium.
H_medium = 6/8 * log2(8/6) + 2/8 * log2(8/2)
H_not_medium = 6/12 * log2(12/6) + 6/12 * log2(12/6)

entropy_medium = 8/20 * (H_medium) + 12/20 * (H_not_medium)

print(entropy_tall, entropy_medium)

0.9280757477080679 0.9245112497836532


You should have found entropies of .925 for medium and .928 for tall. Both of those entropies are higher. Now, we've previously said we want to prioritize questions with the most information gain. Which one of these would that be?

It would be asking if an individual is short. You should also note that for all possible questions, we're still comparing with the same initial entropy value. So one way of seeing which question has the most information gain is to find the one with the lowest entropy. This should make sense when we think of entropy as uncertainty. The less uncertainty after a question, the more information we gained.

## Pseudocoding the Algorithm

**Pseudocode** is the processes of writing the steps and logic you would implement in code, but in normal language rather than commands a programming language could execute. It can be a useful way to chart out how you want to build an algorithm and is a common topic for technical interviews. Here we'll use pseudocode to explain the ID3 algorithm.

Here is reasonable pseudocode for ID3, which we'll then follow up with an explanation of the steps. The outcome for this variable will be A or B. An attribute is denoted a<sub>i</sub>. A value of that attribute is v<sub>i</sub>.


<pre>
Algorithm(Observations, Outcome, Attributes)
    Create a root node.
    If all observations are 'A', label root node 'A' and return.
    If all observations are 'B', label root node 'B' and return.
    If no attributes return the root note labeled with the most common Outcome.
    Otherwise, start:
        For each value v<sub>i</sub> of each attribute a<sub>i</sub>, calculate the entropy.
        The attribute a<sub>i</sub> and value v<sub>i</sub> with the lowest entropy is the best rule.
        The attribute for this node is then a<sub>i</sub>
            Split the tree to below based on the rule a<sub>i</sub> = v<sub>i</sub>
            Observations<sub>v<sub>i</sub></sub> is the subset of observations with value v<sub>i</sub>
            If Observations<sub>v<sub>i</sub></sub> is empty cap with node labeled with most common Outcome
            Else at the new node start a subtree (Observations<sub>v<sub>i</sub></sub>, Target Outcome, Attributes - {a<sub>i</sub>}) and repeat the algorithm
</pre>

Now let's walk through this pseudocode algorithm in plain English piece by piece.

First you create a root node. Simple enough, you have to start with something.

The next two lines say that if you're already exclusively one class, just label with that class and you're done. Similarly the following line says that if you don't have any attributes left you're also done, labeling with the most common outcome.

Then we get into the _real_ algorithm. First you have to find the best attribute by calculating entropies for all possible values. The attribute value pair with the lowest entropy is then the best attribute and the attribute you give to the node.

You use that rule to split the node, creating subsets of observations. There are then two new nodes, each with a subset of the observations corresponding to the rule. If obsevations are null then label with the dominant outcome.

Otherwise at each new node start the algorithm again.

This is how a decision tree would actually function. Understanding this should give you some insight into how this algorithm works and reveals several attributes of the algorithm. Firstly, the solution is not necessarily optimal. The tree can get stuck in local optima, just like with the gradient descent algorithm. It also has no way to work backwards if it finds itself in an informationless space. You can add criteria to make it stop before the tree has grown to run out of attributes or all leaf nodes are single classes.

## DRILL:

Look over the code for [this real](https://github.com/NinjaSteph/DecisionTree/blob/master/sna2111_DecisionTree/DecisionTree.py) ID3 Algorithm in python. Note how well the author breaks up functionality into individual, reusable, well-named helper functions. See if you can match our pseudocode steps to the code in this example.